In [1]:
!pip install docx2txt

  Preparing metadata (setup.py) ... done
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl size=3960 sha256=595c42b0120cfdcd4faf9fd35dfe45cd41dce6781a222e91aef7c2142c49ffc8
  Stored in directory: /root/.cache/pip/wheels/22/58/cf/093d0a6c3ecfdfc5f6ddd5524043b88e59a9a199cb02352966
Successfully built docx2txt


In [2]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 2.0 MB/s eta 0:00:00


In [3]:
import os
import docx2txt
from docx import Document
from google.colab import files

In [4]:
# Function to upload files to Colab
def upload_files():
    uploaded = files.upload()
    return list(uploaded.keys())

In [5]:
# Function to convert folder to text and save as new Word documents
def convert_folder_to_text(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Upload files to Colab
    uploaded_files = upload_files()

    # Iterate through each uploaded file
    for filename in uploaded_files:
        if filename.endswith(".docx"):
            # Extract text from the uploaded Word document
            text = docx2txt.process(filename)

            # Create a new Word document and add the extracted text
            doc = Document()
            doc.add_paragraph(text)

            # Save the new Word document to the output folder
            output_filepath = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_output.docx")
            doc.save(output_filepath)

In [11]:
if __name__ == "__main__":
    input_folder_path = "/content/input_folder"
    output_folder_path = "/content/output_folder"

    # Convert uploaded files to text and save as new Word documents
    convert_folder_to_text(input_folder_path, output_folder_path)

Saving Roya SAP_LA_SDD-PM-09_Corrective_Maintenance_V1.0.docx to Roya SAP_LA_SDD-PM-09_Corrective_Maintenance_V1.0.docx


In [7]:
!pip install cohere tiktoken
!pip install h11 httpcore httpx openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 782.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.7 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib-metadata 7.0.0
    Uninstalling importlib-metadata-7.0.0:
      Successfully uninstalled importlib-metadata-7.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires openai, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 606.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 4.3 MB/s eta 0:00:00


In [8]:
!pip install --upgrade openai

In [9]:
!pip show openai

Name: openai
Version: 1.5.0
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: llmx


In [14]:
from openai import OpenAI
import json

api_key = api_key
client = OpenAI(api_key=api_key)


def convert_to_gpt35_format(document_path):
    data = {
        "messages": [
            {
                "role": "system",
                "content": "This is Digi-Assist"
            },
            {
                "role": "user",
                "content": "Generate a Business solution design document for corrective maintenance process"
            },
            {
                "role": "assistant",
                "content": open(document_path, "rb").read().decode("latin-1")
            }
        ]
    }
    return data

# Load the docx file
docx_file = "/content/output_folder/Roya SAP_LA_SDD-PM-09_Corrective_Maintenance_V1.0_output.docx"
data_dict = convert_to_gpt35_format(docx_file)

# Convert data to JSONL format
jsonl_content = json.dumps(data_dict) + "\n"

# Choose output file path
output_file_path = "fine_tuning_data.jsonl"

# Write JSONL data to file
with open(output_file_path, "w") as f:
    f.write(jsonl_content)

# Upload the file object
with open(output_file_path, "rb") as f:
    training_file = client.files.create(file=f, purpose="fine-tune")

# Extract the file ID from the upload response
file_id = training_file.id
print(file_id)

# Create the fine-tuning job with the file ID
suffix_name = "doc-generator"
fine_tuning_job = client.fine_tuning.jobs.create(
    model="gpt-3.5-turbo",
    training_file=file_id,
    suffix = suffix_name
)
id = fine_tuning_job.id

file-smCZnTtxKhLcuCFMoaeD4jNn
FineTuningJob(id='ftjob-bpKNl5VajLhgIaKcWAHu6eUi', created_at=1702903226, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-lfZR7q1zDM1YtptyMFN23gI4', result_files=[], status='validating_files', trained_tokens=None, training_file='file-smCZnTtxKhLcuCFMoaeD4jNn', validation_file=None)
ftjob-bpKNl5VajLhgIaKcWAHu6eUi


In [15]:
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-bpKNl5VajLhgIaKcWAHu6eUi', created_at=1702903226, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-lfZR7q1zDM1YtptyMFN23gI4', result_files=[], status='validating_files', trained_tokens=None, training_file='file-smCZnTtxKhLcuCFMoaeD4jNn', validation_file=None), FineTuningJob(id='ftjob-k7jqMk57vLnO8fCvdcre4UOO', created_at=1702887292, error=Error(code='invalid_n_examples', message='Training file has 1 example(s), but must have at least 10 examples', param='training_file'), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-lfZR7q1zDM1YtptyMFN23gI4', result_files=[], status='failed', trai

In [16]:
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-TF63PWmKLSLbKnXb6ZvU2Sgm", limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-drqe0WFY2pIyGO0SyFnipcXS', created_at=1702864808, level='error', message='Training file has 1 example(s), but must have at least 10 examples', object='fine_tuning.job.event', data={'error_code': 'invalid_n_examples', 'error_param': 'training_file'}, type='message'), FineTuningJobEvent(id='ftevent-z3FEd44eOxDrUKSVhKqjBqzM', created_at=1702864797, level='info', message='Validating training file: file-pNhhD2KqJxQuzlJWVC7Og2bn', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-FuCrmxoHULSybpxB3dwu0K1j', created_at=1702864797, level='info', message='Created fine-tuning job: ftjob-TF63PWmKLSLbKnXb6ZvU2Sgm', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [17]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo-0613",
  messages=[
    {"role": "system", "content": "This is Digi-Assist"},
    {"role": "user", "content": "Generate a document for corrective maintenance process"}
  ]
)
# print(completion.choices[0])


completion = response.choices[0]
text = completion.message.content


print(text)

Corrective Maintenance Process

Objective:
The objective of the corrective maintenance process is to identify and resolve issues and defects in a timely and efficient manner, ensuring the proper functioning of equipment, systems, or processes.

Scope:
The scope of the corrective maintenance process includes identifying and reporting issues, prioritizing and planning maintenance activities, executing repairs or replacements, and documenting the entire process for future reference and analysis.

Process Steps:

1. Issue Identification:
   - Identify and document the issue or defect based on reported problems, observations, or system alerts.
   - Gather relevant information such as location, equipment involved, specific symptoms, and any error messages received.

2. Issue Reporting:
   - Report the identified issue to the maintenance team or designated personnel.
   - Provide a detailed description of the issue, including any supporting documentation or evidence.

3. Issue Prioritization:

In [18]:
import pickle
with open('fine_tuned_job.pkl', 'wb') as f:
    pickle.dump(fine_tuning_job, f)

In [19]:
!pip install spacy

In [20]:
!python -m spacy download en_core_web_sm

2023-12-18 12:42:22.494055: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 12:42:22.494159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 12:42:22.499096: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-18 12:42:22.522061: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-18 12:42:24.507923: W tensorflow/compiler/tf2

In [23]:
from docx import Document
from docx.shared import RGBColor
import spacy

def add_heading(doc, text, level):
    heading = doc.add_heading(text, level=level)
    run = heading.runs[0]
    run.font.color.rgb = RGBColor(0, 0, 0)  # Set color to black

def add_paragraph(doc, text):
    paragraph = doc.add_paragraph(text)
    run = paragraph.runs[0]
    run.font.color.rgb = RGBColor(0, 0, 0)  # Set color to black

def process_text(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)

    document = Document()

    for paragraph in doc.sents:
        # Identify headings based on sentence length
        if len(paragraph) < 50:
            add_heading(document, paragraph.text, level=1)
        else:
            add_paragraph(document, paragraph.text)

    return document

if __name__ == "__main__":
    text = completion.message.content
    # Process the text and create a Word document
    doc = process_text(text)
    # Save the document
    doc.save("output_document.docx")